In [1]:
import pandas as pd
file_path = r'~\Music_recommendation\new_df_mvp'
df_mvp = pd.read_csv(file_path, index_col= [0])
df_mvp.shape

(135420, 21)

In [2]:
#Now I'm trying Spotify API
#Optionally, create env var for security later
client_id= 'ed0dee5af2274cebb2f1564a116b2bcf'
client_secret = '90fcfc0e15764d9bba8f7b4011345dc1'


In [11]:
#!pip install spotipy
import spotipy # spotipy wrapper package
from spotipy.oauth2 import SpotifyClientCredentials # for authentication

import numpy as np
#get the unique values for track_ids
unique_track_ids = df_mvp['track_id'].unique()

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id,
                                                           client_secret=client_secret))

def search_spotify(q:str):
    global search_results # accessible globally in this notebook
    search_results = sp.search(q=q, limit=1) 
    if search_results['tracks']['items']:
        #track_name = search_results['tracks']['items'][0]['name']
        #album_name = search_results['tracks']['items'][0]['album']['name']
        #artist_name = search_results['tracks']['items'][0]['artists'][0]['name']
        #popularity = search_results['tracks']['items'][0]['popularity']
        artist_id = search_results['tracks']['items'][0]['artists'][0]['id']
        if artist_id:
            artist = sp.artist(artist_id)
            genres = artist['genres']
            
            return {
                'genres': ', '.join(genres),
                'track_id': q
            }
        else:
            # Return NaN for genres if artist_id is not valid
            return {
                'genres': np.nan,
                'track_id': q
            }
        
    else:
        # Return a dictionary with NaN values if no track is found
        return {
            #'track_name': np.nan,
            #'album_name': np.nan,
            #'artists': np.nan,
            #'popularity': np.nan,
            'genres' :np.nan,
            'track_id': q
        }

results_list = []
i=0
#Check tracks, artists and albums,popularity
for track_id in unique_track_ids[15000:20000]:
   result = search_spotify(track_id)
   print('hello', i)
   results_list.append(result)
   i += 1
   
df_genres = pd.DataFrame(results_list)

In [6]:
df_genres.to_csv(r'~\Music_recommendation\new_tracks\df_genres_10000', index= True)

In [54]:
#Info is taken in batches from Spotify API(df1,df2,df3)

df_1 = pd.read_csv(r'~\Music_recommendation\df_results_10000', index_col=[0])
df_2 = pd.read_csv(r'~\Music_recommendation\df_results_23000', index_col=[0])
df_3 = pd.read_csv(r'~\Music_recommendation\df_results_last', index_col=[0])
df_new = pd.concat([df_1, df_2], ignore_index=True)
df_resulted = pd.concat([df_new, df_3], ignore_index=True)
df_resulted.to_csv(r'~\Music_recommendation\new_tracks\df_mvp_tracks', index= True)


In [3]:
tracks = pd.read_csv(r'~\Music_recommendation\new_tracks\df_mvp_tracks', index_col=[0])
tracks
#tracks.loc[tracks.track_id =='42ffc105744c630858cdf48680ed21dc']

,track_name,album_name,artists,popularity,track_id
0,It's Corn,It's Corn,Tariq,53.0,8f2ac86abb8bd48273c8fc95b632e347
1,Crazy Rap (Colt 45) - GAYMIX,Crazy Rap (Colt 45),DigBar,48.0,45bbd6d7cd65dc77596af8c5c0b89a70
2,Dance Mode,Dance Mode!,Bluey,64.0,d53b2de022666067050dae8f6645edc2
3,abc,KIDZ BOP Super POP!,Kidz Bop Kids,48.0,2ac754eb153631a50e64b0b37801ef21
4,I'm Good (Blue),KIDZ BOP 2023,Kidz Bop Kids,53.0,40cbffe332ab6b773121e1fbfa798e1b
...,...,...,...,...,...
28867,1539,1539,KeyKey,40.0,1495d614085ef45b87d6b3f1fc8d0a34
28868,Six in a Bed,50 Silly Songs,The Little Sunshine Kids,64.0,6ec1b24c1a5f2982ad971bbf62e7ae89
28869,╜φ°⌂▌╫§╜φ°⌂▌╫§╜φ°⌂▌╫§╜φ°⌂▌╫§╜φ°⌂▌╫§╜φ°⌂▌╫§╜φ°⌂...,Hi This Is Flume (Mixtape),Flume,37.0,960f037c161aa000cf5df179d1bf4c3c
28870,It's Corn,It's Corn,Tariq,53.0,33c90e87f7256db3fb776e2133c30309


In [9]:
#This is the new df with track_names, albums, artists and popularity included
df_mvp_new = pd.read_csv(r'~\Music_recommendation\new_tracks\df_mvp_new', index_col=[0])


In [4]:
genre_1 = pd.read_csv(r'~\Music_recommendation\new_tracks\df_genres_5000.csv', index_col=[0])
genre_2 = pd.read_csv(r'~\Music_recommendation\new_tracks\df_genres_10000.csv', index_col=[0])
genre_3 = pd.read_csv(r'~\Music_recommendation\new_tracks\df_genres_15000.csv', index_col=[0])

genre_new = pd.concat([genre_1, genre_2, genre_3], ignore_index=True)
df_genres= pd.merge(tracks, genre_new, on='track_id', how='left')


In [7]:
#After a long time spending on the Spotify API, it's decided to fetch the remaining genres(28872-15000) with matching Album Names
#Check how many unique album_name we have, then complete the NAN genres. 

df_genres.album_name.nunique()
#2225 albums
for index, row in df_genres.iterrows():
    if pd.isna(row['genres']):
        result = df_genres.loc[(df_genres['album_name'] == row['album_name']) & ~df_genres['genres'].isna(), 'genres']
    if not result.empty:
        df_genres.at[index, 'genres'] = result.iloc[0]    


In [29]:
#Add the genres to the df_mvp dataset
df_result= pd.merge(df_mvp, df_genres, on='track_id', how='left')
df_result.to_csv(r'~\Music_recommendation\new_tracks\df_mvp_result.csv')
